### IMPORTING MODULES

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

### DATA PREPARATION

In [ ]:
import os
path = os.listdir('CK+48')

In [ ]:
classes = {'anger':0,'contempt':1, 'disgust':2, 'fear':3, 'happy':4, 'sadness':5, 'surprise':6}

In [ ]:
import cv2
X = []
Y = []
for cls in classes:
    pth = 'CK+48/'+cls
    for j in os.listdir(pth):
        img = cv2.imread(pth+'/'+j,0)
        img = cv2.resize(img, (200,200))
        X.append(img)
        Y.append(classes[cls])

In [ ]:
X=np.array(X)
Y=np.array(Y)

In [ ]:
X_updated = X.reshape(len(X),-1)

In [ ]:
np.unique(Y)

In [ ]:
pd.Series(Y).value_counts()

In [ ]:
X.shape, X_updated.shape

### DATA VISUALIZATION

In [ ]:
plt.imshow(X[0], cmap='gray')

In [ ]:
X_updated = X.reshape(len(X), -1)
X_updated.shape

### DATA SPLITTING

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(X_updated, Y, random_state=10,test_size=.20)

In [ ]:
xtrain.shape, xtest.shape

### FEATURE SCALING

In [ ]:
print(xtrain.max(), xtrain.min())
print(xtest.max(), xtest.min())
xtrain = xtrain/255
xtest = xtest/255
print(xtrain.max(), xtrain.min())
print(xtest.max(), xtest.min())

### FEATURE SELECTION: USING PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
print(xtrain.shape, xtest.shape)

pca = PCA(.98)
pca_train = xtrain
pca_test = xtest

### TRAINING MODEL USING SVM AND LR


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [ ]:
import warnings
warnings.filterwarnings('ignore')

lg = LogisticRegression(C=0.1)
lg.fit(xtrain, ytrain)

In [ ]:
sv = SVC()
sv.fit(xtrain, ytrain)

### EVALUATION

In [ ]:
print("Training Score:", lg.score(xtrain, ytrain))
print("Testing Score:", lg.score(xtest, ytest))

In [ ]:
print("Training Score:", sv.score(xtrain, ytrain))
print("Testing Score:", sv.score(xtest, ytest))

### PREDICTION

In [ ]:
# using SVM

pred = sv.predict(xtest)

In [ ]:
# using Logistic Regression

pred2 = lg.predict(xtest)

In [ ]:
misclassified=np.where(ytest!=pred)
misclassified


In [ ]:
print("Total Misclassified Samples: ",len(misclassified[0]))
print(pred[36],ytest[36])

In [ ]:
dec = {0:'anger', 1:'contempt', 2:'disgust', 3:'fear', 4:'happy', 5:'sadness', 6:'surprise'}

### TESTING MODEL

#### USING SVM

In [ ]:
plt.figure(figsize=(12,8))
p = os.listdir('CK+48/TestImages/')
c=1
for i in os.listdir('CK+48/TestImages/imgs/')[:9]:
    plt.subplot(3,3,c)
    
    img = cv2.imread('CK+48/TestImages/imgs/'+i,0)
    img1 = cv2.resize(img, (200,200))
    img1 = img1.reshape(1,-1)/255
    p = sv.predict(img1)
    print(dec[p[0]])
    plt.title(dec[p[0]])
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    c+=1

In [ ]:

from sklearn import metrics


print( 'Accuracy : ' , metrics.accuracy_score(ytest, pred) )
print( 'precision : ' , metrics.precision_score(ytest, pred, average='micro') )
print( 'recall_score : ' , metrics.recall_score(ytest, pred, average='micro') )
print( 'fbeta_score : ' , metrics.fbeta_score(ytest, pred , beta = 0.5 , average='micro') )

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns


cm = confusion_matrix(ytest, pred)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True)
plt.title('Confusion Matrix')
plt.ylabel('Actal Values')
plt.xlabel('Predicted Values')
plt.show()


#### USING LOGISTIC REGRESSION

In [ ]:
plt.figure(figsize=(12,8))
p = os.listdir('CK+48/TestImages/')
c=1
for i in os.listdir('CK+48/TestImages/imgs/')[:9]:
    plt.subplot(3,3,c)
    
    img = cv2.imread('CK+48/TestImages/imgs/'+i,0)
    img1 = cv2.resize(img, (200,200))
    img1 = img1.reshape(1,-1)/255
    p = lg.predict(img1)
    print(dec[p[0]])
    plt.title(dec[p[0]])
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    c+=1

In [ ]:

from sklearn import metrics


print( 'Accuracy : ' , metrics.accuracy_score(ytest, pred2) )
print( 'precision : ' , metrics.precision_score(ytest, pred2, average='micro') )
print( 'recall_score : ' , metrics.recall_score(ytest, pred2, average='micro') )
print( 'fbeta_score : ' , metrics.fbeta_score(ytest, pred2 , beta = 0.5 , average='micro') )

In [ ]:
cm2 = confusion_matrix(ytest, pred2)
plt.figure(figsize=(5,4))
sns.heatmap(cm2, annot=True)
plt.title('Confusion Matrix')
plt.ylabel('Actal Values')
plt.xlabel('Predicted Values')
plt.show()

In [ ]:
from joblib import Parallel, delayed
import joblib


# Save the model as a pickle in a file
joblib.dump(sv, 'filename.pkl')


